In [13]:
########### Python Form Recognizer Analyze Layout #############

import json
import time
from requests import get, post

# Endpoint URL
endpoint = r"https://{ }.cognitiveservices.azure.com/"
apim_key = " "
post_url = endpoint + "/formrecognizer/v2.1/layout/analyze"
source = " " #filename for layout extraction 

headers = {
    # Request headers
    'Content-Type': 'application/pdf', ##### (or should be 'image/jpeg', 'image/png' or 'image/tiff' fepneding on the image type)
    'Ocp-Apim-Subscription-Key': apim_key,
}
with open(source, "rb") as f:
    data_bytes = f.read()

try:
    resp = post(url = post_url, data = data_bytes, headers = headers)
    if resp.status_code != 202:
        print("POST analyze failed:\n%s" % resp.text)
        quit()
    print("POST analyze succeeded:\n%s" % resp.headers)
    get_url = resp.headers["operation-location"]
except Exception as e:
    print("POST analyze failed:\n%s" % str(e))
    quit()

In [5]:
n_tries = 10
n_try = 0
wait_sec = 10
resp_json = ""
while n_try < n_tries:
    try:
        resp = get(url = get_url, headers = {"Ocp-Apim-Subscription-Key": apim_key})
        resp_json = json.loads(resp.text)
        print("Running" )
        if resp.status_code != 200:
            print("GET Layout results failed:\n%s" % resp_json)
            break
        status = resp_json["status"]
        if status == "succeeded":
            print("Succeeded")
            
            #file = open(source.split('.')[0]+"_jsonoutput.txt", "w")
            #file.write(json.dumps(resp.json())) 
            #file.close()
            break
        if status == "failed":
            print("Layout Analysis failed:\n%s" % resp_json)
            break
        # Analysis still running. Wait and retry.
        time.sleep(wait_sec)
        n_try += 1     
    except Exception as e:
        msg = "GET analyze results failed:\n%s" % str(e)
        print(msg)
        break

Running
Succeeded


In [1]:
import pandas as pd
from IPython.display import display
import re

pd.options.display.max_columns = None
lines=[]
for read_result in resp_json["analyzeResult"]["readResults"]:
    # print("Page No:%s" % read_result["page"])
    print("--------------Page %d: Extracted OCR---------------" % read_result["page"])
    for line in read_result["lines"]:
        lines.append((line["text"]))

for pageresult in resp_json["analyzeResult"]["pageResults"]:
    # print("Page No:%s" % pageresult["page"])
    #if pageresult["page"] == 4:
   
    for table in pageresult['tables']:
        print("--------------Page %d: Extracted table---------------" % pageresult["page"])
        # print("-------------------Extracted Table-------------------")
        print("No of Rows: %s" % table["rows"])
        print("No of Columns: %s" % table["columns"])
        tableList = [[None for x in range(table["columns"])] for y in range(table["rows"])] 
        print(type(tableList))
        for cell in table['cells']:
            if 'columnSpan' not in cell.keys():
                tableList[cell["rowIndex"]][cell["columnIndex"]] = cell["text"]
                #print(cell["rowIndex"],cell["columnIndex"])
            elif cell['columnSpan']==2:
                #print(cell['columnSpan'],cell["text"])
                #print(re.split(', |!| |\|', cell["text"])[0])
                #print(cell["rowIndex"],cell["columnIndex"])
                tableList[cell["rowIndex"]][cell["columnIndex"]]=re.split(',|!| |\|', cell["text"])[0]
                
                cell["columnIndex"]=cell["columnIndex"]+1
                tableList[cell["rowIndex"]][cell["columnIndex"]]=' '.join(cell["text"].split(' ')[1:])
           
                
        #print("new table" , tableList)
        df = pd.DataFrame.from_records(tableList)
        display(df)
        df.to_csv("extractions_"+source.split('.')[0]+"_"+str(pageresult["page"])+".csv")
        #print(df)


In [2]:
print(resp_json)
